In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("Extended_Coffee_Data.csv")

In [3]:
df

,transaction_date,product_id,transaction_qty,product_category
0,01-01-2023,67,3,Flavours
1,02-01-2023,17,7,Bakery
2,03-01-2023,9,7,Drinking Chocolate
3,04-01-2023,97,15,Coffee
4,05-01-2023,30,4,Branded
...,...,...,...,...
3595,17-12-2023,61,4,Tea
3596,18-12-2023,71,25,Coffee
3597,19-12-2023,63,2,Packaged Chocolate
3598,20-12-2023,46,4,Branded


In [4]:
import pandas as pd

df = pd.read_csv("Extended_Coffee_Data.csv")

# har category ko ek unique numeric id
df["product_id"] = df["product_category"].astype("category").cat.codes + 1

print(df.head())


  transaction_date  product_id  transaction_qty    product_category
0       01-01-2023           6                3            Flavours
1       02-01-2023           1                7              Bakery
2       03-01-2023           5                7  Drinking Chocolate
3       04-01-2023           3               15              Coffee
4       05-01-2023           2                4             Branded


In [5]:
df

,transaction_date,product_id,transaction_qty,product_category
0,01-01-2023,6,3,Flavours
1,02-01-2023,1,7,Bakery
2,03-01-2023,5,7,Drinking Chocolate
3,04-01-2023,3,15,Coffee
4,05-01-2023,2,4,Branded
...,...,...,...,...
3595,17-12-2023,9,4,Tea
3596,18-12-2023,3,25,Coffee
3597,19-12-2023,8,2,Packaged Chocolate
3598,20-12-2023,2,4,Branded


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller


In [7]:
!pip install statsmodels


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
df["transaction_date"] = pd.to_datetime(
    df["transaction_date"],
    format="%d-%m-%Y"
)


In [9]:
df

,transaction_date,product_id,transaction_qty,product_category
0,2023-01-01,6,3,Flavours
1,2023-01-02,1,7,Bakery
2,2023-01-03,5,7,Drinking Chocolate
3,2023-01-04,3,15,Coffee
4,2023-01-05,2,4,Branded
...,...,...,...,...
3595,2023-12-17,9,4,Tea
3596,2023-12-18,3,25,Coffee
3597,2023-12-19,8,2,Packaged Chocolate
3598,2023-12-20,2,4,Branded


In [31]:
monthly = (
    df.groupby([
        df["transaction_date"].dt.to_period("M"),
        "product_category"
    ])["transaction_qty"]
    .sum()
    .reset_index()
)

monthly["transaction_date"] = monthly["transaction_date"].dt.to_timestamp()

,transaction_date,product_category,transaction_qty
0,2023-01-01,Bakery,108
1,2023-01-01,Branded,145
2,2023-01-01,Coffee,395
3,2023-01-01,Coffee beans,187
4,2023-01-01,Drinking Chocolate,141
...,...,...,...
103,2023-12-01,Drinking Chocolate,118
104,2023-12-01,Flavours,145
105,2023-12-01,Loose Tea,107
106,2023-12-01,Packaged Chocolate,124


In [11]:
df['transaction_date'].dtype

dtype('<M8[ns]')

In [12]:
df["month"] = df["transaction_date"].dt.to_period("M").astype(str)

In [13]:
monthly_cat = (
    df.groupby(["month", "product_category"])["transaction_qty"]
      .sum()
      .reset_index(name="monthly_demand")
)

In [14]:
monthly_cat["month"] = pd.to_datetime(monthly_cat["month"])
monthly_cat["month_number"] = monthly_cat["month"].dt.month

In [15]:
monthly_cat = monthly_cat.sort_values(
    ["product_category", "month"]
)

In [16]:
monthly_cat["prev_month_demand"] = (
    monthly_cat
    .groupby("product_category")["monthly_demand"]
    .shift(1)
)

In [17]:
monthly_cat = monthly_cat.sort_values(
    ["product_category", "month"]
)

monthly_cat["avg_last_3_months"] = (
    monthly_cat
    .groupby("product_category")["monthly_demand"]
    .shift(1)
    .rolling(3)
    .mean()
)


In [18]:
monthly_cat["next_month_demand"] = (
    monthly_cat
    .groupby("product_category")["monthly_demand"]
    .shift(-1)
)


In [19]:
model_df = monthly_cat.dropna().reset_index(drop=True)
model_df.head()

,month,product_category,monthly_demand,month_number,prev_month_demand,avg_last_3_months,next_month_demand
0,2023-04-01,Bakery,123,4,139.0,131.000000,94.0
1,2023-05-01,Bakery,94,5,123.0,136.000000,93.0
2,2023-06-01,Bakery,93,6,94.0,118.666667,129.0
3,2023-07-01,Bakery,129,7,93.0,103.333333,143.0
4,2023-08-01,Bakery,143,8,129.0,105.333333,121.0


In [21]:
from catboost import CatBoostRegressor

FEATURES = [
    "month_number",
    "prev_month_demand",
    "avg_last_3_months",
    "product_category"
]

TARGET = "next_month_demand"

X = model_df[FEATURES]
y = model_df[TARGET]

cat_features = [X.columns.get_loc("product_category")]

In [22]:
model = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    loss_function="RMSE",
    verbose=False
)

model.fit(X, y, cat_features=cat_features)

In [32]:
def predict_for_month(input_month_number):
    rows = []

    for cat in model_df["product_category"].unique():
        cat_data = monthly_cat[monthly_cat["product_category"] == cat]

        prev_month_demand = cat_data.iloc[-1]["monthly_demand"]
        avg_last_3 = cat_data.tail(3)["monthly_demand"].mean()

        rows.append({
            "month_number": input_month_number,
            "prev_month_demand": prev_month_demand,
            "avg_last_3_months": avg_last_3,
            "product_category": cat
        })

    pred_df = pd.DataFrame(rows)
    pred_df["predicted_demand"] = model.predict(pred_df)

    return pred_df


In [33]:
july_pred = predict_for_month(7)
july_pred

,month_number,prev_month_demand,avg_last_3_months,product_category,predicted_demand
0,7,115,150.000000,Bakery,122.418657
1,7,148,146.333333,Branded,132.011272
2,7,709,505.000000,Coffee,110.189732
3,7,120,106.666667,Coffee beans,239.104456
4,7,118,130.666667,Drinking Chocolate,131.575121
5,7,145,131.666667,Flavours,145.283757
6,7,107,131.666667,Loose Tea,138.083030
7,7,124,138.666667,Packaged Chocolate,123.658892
8,7,179,153.666667,Tea,116.159446


In [25]:
max_demand = (
    monthly_cat
    .groupby("product_category")["monthly_demand"]
    .max()
    .to_dict()
)

In [26]:
july_pred["stock_percent"] = (
    july_pred["predicted_demand"] /
    july_pred["product_category"].map(max_demand)
) * 100

In [27]:
category_product_map = (
    df[["product_id", "product_category"]]
    .drop_duplicates()
)

In [28]:
category_product_map

,product_id,product_category
0,6,Flavours
1,1,Bakery
2,5,Drinking Chocolate
3,3,Coffee
4,2,Branded
5,8,Packaged Chocolate
6,7,Loose Tea
16,9,Tea
20,4,Coffee beans


In [29]:
final_output = july_pred.merge(
    category_product_map,
    on="product_category",
    how="left"
)

final_output = final_output[
    ["product_id", "product_category", "stock_percent"]
]

final_output

,product_id,product_category,stock_percent
0,1,Bakery,70.248475
1,2,Branded,79.937826
2,3,Coffee,15.574030
3,4,Coffee beans,129.446284
4,5,Drinking Chocolate,75.851477
5,6,Flavours,95.136019
6,7,Loose Tea,77.593184
7,8,Packaged Chocolate,74.503921
8,9,Tea,57.402239
